‘BIO’ labels. Note that the ‘BIO’ labels are considered the hidden states, whereas the vocabulary tokens are the observations.

In [39]:
import numpy as np
from collections import defaultdict
import pandas as pd
import pickle


In [9]:
df = pd.read_csv("/Users/indusha-22955/Documents/Learning/NER dataset.csv",encoding="latin1")
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [11]:
df = df.fillna(method="ffill")
df.head()

/var/folders/8s/n5hd259j209817bmcvswgr4m0000gp/T/ipykernel_61076/2523400444.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [15]:
agg_func = lambda s: [
(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                s["POS"].values.tolist(),
                                s["Tag"].values.tolist())
]

grouped = df.groupby("Sentence #").apply(agg_func) 
sentences = [s for s in grouped]

/var/folders/8s/n5hd259j209817bmcvswgr4m0000gp/T/ipykernel_61076/2801204159.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby("Sentence #").apply(agg_func)


In [16]:
print(sentences[0])

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [17]:
len(sentences)

47959

In [27]:
words_all = [w[0].lower() for s in sentences for w in s]
print(len(words_all))
words = list(set(words_all))
print(len(words))

#how are words more than sentences number?


1048575
31817


In [ ]:
words.append("ENDPAD")

tags = list(set([w[2] for s in sentences for w in s]))

word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2word = {i: w for w, i in word2idx.items()}
idx2tag = {i: t for t, i in tag2idx.items()}

print(len(word2idx),len(tag2idx))

17
31818 17


In [29]:
sentences[-1]

[('The', 'DT', 'O'),
 ('United', 'NNP', 'B-org'),
 ('Nations', 'NNP', 'I-org'),
 ('is', 'VBZ', 'O'),
 ('praising', 'VBG', 'O'),
 ('the', 'DT', 'O'),
 ('use', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('military', 'JJ', 'O'),
 ('helicopters', 'NNS', 'O'),
 ('to', 'TO', 'O'),
 ('drop', 'VB', 'O'),
 ('food', 'NN', 'O'),
 ('and', 'CC', 'O'),
 ('rescue', 'NN', 'O'),
 ('survivors', 'NNS', 'O'),
 ('in', 'IN', 'O'),
 ('tsunami-ravaged', 'JJ', 'B-tim'),
 ('Indonesia', 'NNP', 'I-tim'),
 (',', ',', 'O'),
 ('saying', 'VBG', 'O'),
 ('the', 'DT', 'O'),
 ('aircraft', 'NN', 'O'),
 ('are', 'VBP', 'O'),
 ('"', '``', 'O'),
 ('worth', 'IN', 'O'),
 ('their', 'PRP$', 'O'),
 ('weight', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('gold', 'NN', 'O'),
 ('.', '.', 'O'),
 ('"', '``', 'O')]

In [30]:
def hmm(sentences, word2idx, tag2idx):
    """Trains the HMM model."""

    n_words = len(word2idx)
    n_tags = len(tag2idx)

    transition_counts = np.zeros((n_tags, n_tags))
    emission_counts = np.zeros((n_tags, n_words))
    initial_counts = np.zeros(n_tags)

    for sentence in sentences:
        initial_counts[tag2idx[sentence[0][2]]] += 1  
        for i in range(len(sentence) - 1):
            current_tag = sentence[i][2]
            next_tag = sentence[i+1][2]
            word = sentence[i][0].lower() 
            transition_counts[tag2idx[current_tag], tag2idx[next_tag]] += 1 #trans[2][3]
            emission_counts[tag2idx[current_tag], word2idx[word]] += 1

    
        last_word = sentence[-1][0].lower()
        last_tag = sentence[-1][2]
        emission_counts[tag2idx[last_tag], word2idx[last_word]] +=1

    alpha = 0.0005 # Laplace
    transition_probabilities = (transition_counts + alpha) / (np.sum(transition_counts, axis=1, keepdims=True) + alpha * n_tags) # (state -> state)
    emission_probabilities = (emission_counts + alpha) / (np.sum(emission_counts, axis=1, keepdims=True) + alpha * n_words)   # (state -> word)
    initial_probabilities = (initial_counts + alpha) / (np.sum(initial_counts) + alpha * n_tags)

    return transition_probabilities, emission_probabilities, initial_probabilities

In [ ]:
"""pay a closer look to this bef explaining to anyone, you're very much confused here"""



def viterbi_decode(words, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag):
    """optimize finding of likely seq"""
    n_words = len(word2idx)
    n_tags = len(tag2idx)
    T = len(words)


    viterbi = np.zeros((n_tags, T)) #matrix
    backpointers = np.zeros((n_tags, T), dtype=int)
    """viterbi[i, t] stores the probability of the most likely sequence ending at word t with tag i.
backpointers[i, t] stores the index of the best previous tag that led to i at time t."""


    for s in range(n_tags): #for each tag we are calculating the prob for the word to find the initial prob of first word tag
        word = words[0].lower() 

        if word in word2idx:
            viterbi[s, 0] = initial_probabilities[s] * emission_probabilities[s, word2idx[word]]
        else:
            viterbi[s, 0] = initial_probabilities[s] * 0.00001 # small prob


    # Recursion step
    for t in range(1, T):#from the second word onward
        word = words[t].lower() 
        for s in range(n_tags): #for each possible tag
            if word in word2idx:
                emission_prob = emission_probabilities[s, word2idx[word]]
            else:
                emission_prob = 0.00001

            viterbi[s, t] = np.max(viterbi[:, t-1] * transition_probabilities[:, s] * emission_prob)
            backpointers[s, t] = np.argmax(viterbi[:, t-1] * transition_probabilities[:, s]) #stores the best possible path at any pt of time

    best_path = [None] * T
    best_path_pointer = np.argmax(viterbi[:, T-1])
    best_path[T-1] = idx2tag[best_path_pointer]

    for t in range(T-2, -1, -1):
        best_path_pointer = backpointers[best_path_pointer, t+1]
        best_path[t] = idx2tag[best_path_pointer]

    return best_path

In [32]:
transition_probabilities, emission_probabilities, initial_probabilities = hmm(sentences[:30000], word2idx, tag2idx)


In [33]:

def evaluate_model(sentences, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag):
    """Evaluates the trained HMM model."""
    correct_predictions = 0
    total_predictions = 0

    for sentence in sentences:
        words = [w[0] for w in sentence]
        true_tags = [w[2] for w in sentence]

        predicted_tags = viterbi_decode(words, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag)

        for i in range(len(true_tags)):
            if predicted_tags[i] == true_tags[i]:
                correct_predictions += 1
            total_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

In [36]:
accuracy = evaluate_model(sentences[30000:], transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag)
print(f"Accuracy: {accuracy}")



Accuracy: 0.9410974644434799


In [37]:
example_sentence = ["Indusha", "went", "to", "New", "Delhi", "."]
predicted_tags = viterbi_decode(example_sentence, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag)
print(f"Sentence: {example_sentence}")
print(f"Predicted Tags: {predicted_tags}")

Sentence: ['Indusha', 'went', 'to', 'New', 'Delhi', '.']
Predicted Tags: ['O', 'O', 'O', 'B-geo', 'I-geo', 'O']


In [ ]:
# Save the model
def save_model(filename,transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag):
    model = {
        'transition_probabilities': transition_probabilities,
        'emission_probabilities': emission_probabilities,
        'initial_probabilities': initial_probabilities,
        'word2idx': word2idx,
        'tag2idx':tag2idx,
        'idx2tag': idx2tag
    }
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    print("Model saved successfully!")

# Save the model
save_model("ner_hmm_model_scratch.pkl", transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag)

Model saved successfully!


In [ ]:
#for future use
def load_pickle(filename="word_embeddings.pkl"):
    """Loads word embeddings and vocabulary from a file."""
    try:
        with open(filename, 'rb') as f:  # 'rb' for read binary
            data = pickle.load(f)
        return data["transition_probabilities"], data['emission_probabilities'], data['initial_probabilities'], data['word2idx'], data['tag2idx'], data['idx2tag']
    except FileNotFoundError:
        print(f"Error: File {filename} not found.")
        return None, None

In [ ]:
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



def load_data(filepath):
    """Loads the NER dataset from a CSV file and returns a pandas DataFrame."""
    df = pd.read_csv(filepath, encoding="latin1")  # Handle potential encoding issues
    df = df.fillna(method="ffill") # Fill missing values with the previous valid observation
    return df

def preprocess_data(df):
    """Preprocesses the DataFrame, creating sentence groupings."""
    # Group by sentence (sentence boundaries are indicated by repeating sentence ID)
    agg_func = lambda s: [
        (w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                     s["POS"].values.tolist(),
                                     s["Tag"].values.tolist())
    ]

    grouped = df.groupby("Sentence #").apply(agg_func) # sentence is not in the input dataset
    sentences = [s for s in grouped]
    return sentences

def create_mappings(sentences):
    """Creates mappings between words, POS tags, NER tags, and indices."""
    words = list(set([w[0].lower() for s in sentences for w in s]))
    words.append("ENDPAD") #special tag indicating the end of a sentence
    n_words = len(words)

    tags = list(set([w[2] for s in sentences for w in s]))
    n_tags = len(tags)

    # Create mappings
    word2idx = {w: i for i, w in enumerate(words)}
    tag2idx = {t: i for i, t in enumerate(tags)}
    idx2word = {i: w for i, w in enumerate(words)}
    idx2tag = {i: t for i, t in enumerate(tags)}

    return word2idx, tag2idx, idx2word, idx2tag, n_words, n_tags

# HMM Model Training

def hmm_train(sentences, word2idx, tag2idx):
    """Trains the HMM model."""
    # Initialize transition and emission probabilities
    n_words = len(word2idx)
    n_tags = len(tag2idx)

    transition_counts = np.zeros((n_tags, n_tags))
    emission_counts = np.zeros((n_tags, n_words))
    initial_counts = np.zeros(n_tags)

    # Count occurrences
    for sentence in sentences:
        initial_counts[tag2idx[sentence[0][2]]] += 1  # count initial state
        for i in range(len(sentence) - 1):
            current_tag = sentence[i][2]
            next_tag = sentence[i+1][2]
            word = sentence[i][0].lower() # make sure words are lowercase
            transition_counts[tag2idx[current_tag], tag2idx[next_tag]] += 1
            emission_counts[tag2idx[current_tag], word2idx[word]] += 1

        #handle last word in sentence
        last_word = sentence[-1][0].lower()
        last_tag = sentence[-1][2]
        emission_counts[tag2idx[last_tag], word2idx[last_word]] +=1

    # Calculate probabilities with smoothing (Laplace smoothing)
    alpha = 0.0001  # Smoothing parameter
    transition_probabilities = (transition_counts + alpha) / (np.sum(transition_counts, axis=1, keepdims=True) + alpha * n_tags) # (state -> state)
    emission_probabilities = (emission_counts + alpha) / (np.sum(emission_counts, axis=1, keepdims=True) + alpha * n_words)   # (state -> word)
    initial_probabilities = (initial_counts + alpha) / (np.sum(initial_counts) + alpha * n_tags)

    return transition_probabilities, emission_probabilities, initial_probabilities


# Viterbi Decoding
def viterbi_decode(sentence, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag):
    """Decodes the most likely sequence of tags for a given sentence using the Viterbi algorithm."""
    n_words = len(word2idx)
    n_tags = len(tag2idx)
    T = len(sentence)

    # Initialize Viterbi matrix and backpointers
    viterbi = np.zeros((n_tags, T))
    backpointers = np.zeros((n_tags, T), dtype=int)

    # Initialization step
    for s in range(n_tags):
        word = sentence[0].lower()  # make sure words are lowercase

        if word in word2idx:
            viterbi[s, 0] = initial_probabilities[s] * emission_probabilities[s, word2idx[word]]
        else:
             # Handle unknown words using a backoff strategy (e.g., assign a small probability)
            viterbi[s, 0] = initial_probabilities[s] * 0.00001 # small prob


    # Recursion step
    for t in range(1, T):
        word = sentence[t].lower()  # make sure words are lowercase
        for s in range(n_tags):
            if word in word2idx:
                emission_prob = emission_probabilities[s, word2idx[word]]
            else:
                # Handle unknown words
                emission_prob = 0.00001

            viterbi[s, t] = np.max(viterbi[:, t-1] * transition_probabilities[:, s] * emission_prob)
            backpointers[s, t] = np.argmax(viterbi[:, t-1] * transition_probabilities[:, s])

    # Backtracking
    best_path = [None] * T
    best_path_pointer = np.argmax(viterbi[:, T-1])
    best_path[T-1] = idx2tag[best_path_pointer]

    for t in range(T-2, -1, -1):
        best_path_pointer = backpointers[best_path_pointer, t+1]
        best_path[t] = idx2tag[best_path_pointer]

    return best_path


# Evaluation
def evaluate_model(sentences, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag):
    """Evaluates the trained HMM model."""
    correct_predictions = 0
    total_predictions = 0

    for sentence in sentences:
        words = [w[0] for w in sentence]
        true_tags = [w[2] for w in sentence]

        predicted_tags = viterbi_decode(words, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag)

        for i in range(len(true_tags)):
            if predicted_tags[i] == true_tags[i]:
                correct_predictions += 1
            total_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

# Main execution
if __name__ == "__main__":
    # 1. Load and Prepare Data
    data_filepath = "/Users/indusha-22955/Documents/Learning/NER dataset.csv"  # Replace with your actual file path
    df = load_data(data_filepath)
    sentences = preprocess_data(df)

    # 2. Create Mappings
    word2idx, tag2idx, idx2word, idx2tag, n_words, n_tags = create_mappings(sentences)

    # 3. Split Data into Training and Testing Sets
    train_sentences, test_sentences = train_test_split(sentences, test_size=0.2, random_state=42)

    # 4. Train the HMM Model
    transition_probabilities, emission_probabilities, initial_probabilities = hmm_train(train_sentences, word2idx, tag2idx)

    # 5. Evaluate the Model
    accuracy = evaluate_model(test_sentences, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag)
    print(f"Accuracy: {accuracy}")

    # 6. Example Usage (Decoding a New Sentence)
    example_sentence = ["Indusha", "went", "to", "New", "York", "."]
    predicted_tags = viterbi_decode(example_sentence, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag)
    print(f"Sentence: {example_sentence}")
    print(f"Predicted Tags: {predicted_tags}")

/var/folders/8s/n5hd259j209817bmcvswgr4m0000gp/T/ipykernel_61076/605521297.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill") # Fill missing values with the previous valid observation
/var/folders/8s/n5hd259j209817bmcvswgr4m0000gp/T/ipykernel_61076/605521297.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby("Sentence #").apply(agg_func) # sentence is not in the input dataset


Accuracy: 0.945317638087976
Sentence: ['John', 'went', 'to', 'New', 'York', '.']
Predicted Tags: ['B-per', 'O', 'O', 'B-geo', 'I-geo', 'O']


In [21]:
example_sentence = ["cholily", "went", "to", "New", "York", "."]
predicted_tags = viterbi_decode(example_sentence, transition_probabilities, emission_probabilities, initial_probabilities, word2idx, tag2idx, idx2tag)
print(f"Sentence: {example_sentence}")
print(f"Predicted Tags: {predicted_tags}")

Sentence: ['cholily', 'went', 'to', 'New', 'York', '.']
Predicted Tags: ['B-gpe', 'O', 'O', 'B-geo', 'I-geo', 'O']


Standard named entity recognition tags as follows
[
ORGANIZATION - Georgia-Pacific Corp., WHO
PERSON - Eddy Bonte, President Obama
LOCATION - Murray River, Mount Everest
DATE - June, 2008-06-29
TIME - two fifty a m, 1:30 p.m.
MONEY - 175 million Canadian Dollars, GBP 10.40
PERCENT - twenty pct, 18.75 %
FACILITY - Washington Monument, Stonehenge
GPE - South East Asia, Midlothian
]